In [6]:
import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt

def get_dataset_from_testcase(run):
    return run.split('_')[0]

dirs = [
    '/home/chesetti/Repos/LearnedJoinDiskExp/sponge/join_ssd',
    '/home/chesetti/Repos/LearnedJoinDiskExp/sponge/join_threads'
]
runs = []
for dir in dirs:
 for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results')):
        if (run_name=="run"):
           continue
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results',  run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        runs.append(run)
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results', 'run', run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        runs.append(run)
df = pd.json_normalize(runs)
print(df.columns)
display(df)

Index(['command', 'dataset', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.num_output_keys', 'result.outer_disk_fetch',
       'result.outer_disk_fetch_size', 'result.outer_total_bytes_fetched',
       'spec.algo', 'spec.algo_name', 'spec.check_checksum', 'spec.common_key',
       'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk', 'iostat.bytes_read', 'iostat.bytes_wrtn',
       'spec.index.epsilon', 'spec.index.leaf_size_in_pages',
       'spec.index.search', 'spec.index.type'],
      dtype='object')


,command,dataset,result.checksum,result.duration_ns,result.duration_sec,result.inner_disk_fetch,result.inner_disk_fetch_size,result.inner_total_bytes_fetched,result.num_output_keys,result.outer_disk_fetch,...,spec.outer_table,spec.result_path,spec.value_size,spec.write_result_to_disk,iostat.bytes_read,iostat.bytes_wrtn,spec.index.epsilon,spec.index.leaf_size_in_pages,spec.index.search,spec.index.type
0,sponge/join_ssd/build/benchmark_runner sponge/...,fb,0,45841991062,45.841991,781250,4096,3199999984,1989865,0,...,sponge/join_ssd/fb_threads=1/inputs/input100,sponge/join_ssd/fb_threads=1/outputs/hash_join...,8,True,3162272000,28040000,NaN,NaN,NaN,NaN
1,sponge/join_ssd/build/benchmark_runner sponge/...,fb,0,6972632156,6.972632,781250,4096,3199999984,1989865,7773,...,sponge/join_ssd/fb_threads=1/inputs/input100,sponge/join_ssd/fb_threads=1/outputs/sort_join...,8,True,3161960000,64000,NaN,NaN,NaN,NaN
2,sponge/join_ssd/build/benchmark_runner sponge/...,fb,0,30721282928,30.721283,781250,4096,3199999984,199999999,781250,...,sponge/join_ssd/fb_threads=1/inputs/input1,sponge/join_ssd/fb_threads=1/outputs/sort_join...,8,True,6255952000,2565744000,NaN,NaN,NaN,NaN
3,sponge/join_ssd/build/benchmark_runner sponge/...,fb,0,82335846276,82.335846,781250,4096,3199999984,199999999,0,...,sponge/join_ssd/fb_threads=1/inputs/input1,sponge/join_ssd/fb_threads=1/outputs/hash_join...,8,True,6256952000,3094308000,NaN,NaN,NaN,NaN
4,sponge/join_ssd/build/benchmark_runner sponge/...,fb,0,46073187735,46.073188,781250,4096,3199999984,1989865,0,...,sponge/join_ssd/fb_threads=1/inputs/input100,sponge/join_ssd/fb_threads=1/outputs/hash_join...,8,True,3163004000,29896000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,sponge/join_threads/build/benchmark_runner spo...,fb,0,5861108854,5.861109,720771,4096,2952278000,1990024,7776,...,sponge/join_threads/fb_threads=6/inputs/input100,sponge/join_threads/fb_threads=6/outputs/inlj_...,8,True,3169872000,72000,256.0,1.0,binary,btree
548,sponge/join_threads/build/benchmark_runner spo...,fb,0,8615957281,8.615957,7664,69632,533606384,33333336,781254,...,sponge/join_threads/fb_threads=6/inputs/input1,sponge/join_threads/fb_threads=6/outputs/lsj_s...,8,True,3648740000,108000,4096.0,NaN,binary,sampledflatpgm4096
549,sponge/join_threads/build/benchmark_runner spo...,fb,0,1131299183,1.131299,64459,8192,528046064,331673,7776,...,sponge/join_threads/fb_threads=6/inputs/input100,sponge/join_threads/fb_threads=6/outputs/lsj_s...,8,True,557784000,0,256.0,NaN,binary,sampledflatpgm256
550,sponge/join_threads/build/benchmark_runner spo...,fb,0,6544052997,6.544053,1072582,20480,21966446512,19033523,74352,...,sponge/join_threads/fb_threads=6/inputs/input10,sponge/join_threads/fb_threads=6/outputs/inlj_...,8,True,3424912000,500000,1024.0,NaN,binary,sampledflatpgm1024


In [8]:

epsilon = {
    "pgm256": 256,
    "pgm1024": 1024,
    "pgm2048": 2048,
    "flatpgm256": 256,
    "flatpgm1024": 1024,
    "flatpgm2048": 2048,
    "flatpgm4096": 4096,
    "flatpgm8192": 8192,
    "hashjoin": 0,
    "btree": 0,
    "sj": 0,
    "sj2": 0,
    "standard_merge": 0
};
def indexType(x):
    if "pgm" in x:
        return "PGM"
    elif "btree" in x:
        return "BTREE"
    return "NA"

df["threads"] = df["spec.num_threads"]
df["duration_sec"] = df["result.duration_ns"] / (1000000000)
df["inner_disk_fetch"] = df["result.inner_disk_fetch"]
df['result_num_keys'] = df['result.num_output_keys']
df["ratio"] = df["spec.common_key"]
df["algo"] = df["spec.algo_name"]
df["epsilon"] = df["spec.index.epsilon"]
df["inner_bytes_fetched"] = df["result.inner_total_bytes_fetched"]
df["index_type"] = df["spec.algo_name"].map(lambda x: indexType(x))
df['disk_read_gb'] = df['iostat.bytes_read']/1e9
df['disk_wrtn_gb'] = df['iostat.bytes_wrtn']/1e9
#df['cdisk_read_gb'] = df['cgroup_iostat.bytes_read']/1e9
#df['cdisk_wrtn_gb'] = df['cgroup_iostat.bytes_wrtn']/1e9
df['checksum'] = df['result.checksum']
df['threads'] = df['spec.num_threads']

df['dataset'].unique()
df['result.num_output_keys'].unique()
duckdb.sql("SELECT * FROM df WHERE dataset='fb'")


┌──────────────────────┬─────────┬─────────────────┬────────────────────┬───┬──────────────┬──────────────┬──────────┐
│       command        │ dataset │ result.checksum │ result.duration_ns │ … │ disk_read_gb │ disk_wrtn_gb │ checksum │
│       varchar        │ varchar │      int64      │       int64        │   │    double    │    double    │  int64   │
├──────────────────────┼─────────┼─────────────────┼────────────────────┼───┼──────────────┼──────────────┼──────────┤
│ sponge/join_ssd/bu…  │ fb      │               0 │        45841991062 │ … │     3.162272 │      0.02804 │        0 │
│ sponge/join_ssd/bu…  │ fb      │               0 │         6972632156 │ … │      3.16196 │      6.4e-05 │        0 │
│ sponge/join_ssd/bu…  │ fb      │               0 │        30721282928 │ … │     6.255952 │     2.565744 │        0 │
│ sponge/join_ssd/bu…  │ fb      │               0 │        82335846276 │ … │     6.256952 │     3.094308 │        0 │
│ sponge/join_ssd/bu…  │ fb      │              

In [21]:
def print_for_dataset(dataset, algos, ratio):
    print("## " + dataset)
    algo_filter = ''
    for algo in algos:
        algo_filter = algo_filter + f" OR algo='{algo}'"
    algo_filter = algo_filter[3:]
    rows = duckdb.sql(
        " SELECT threads, ratio, algo, MEDIAN(duration_sec) as v FROM df"
        f" WHERE dataset='{dataset}' AND ratio={ratio}"
        f" AND ({algo_filter})"
        " GROUP BY dataset, threads, ratio, algo, threads"
        " ORDER BY ratio, threads"
    ).df()
    rows = rows.pivot(index='threads', values='v', columns='algo')
    print(rows.to_markdown())
    rows[algos].to_csv('/home/chesetti/Repos/LearnedJoinDiskExp/experiments/results/join_thread/'+str(ratio)+'.txt')



algos = ['lsj_sampledflatpgm256', 'inlj_btree256', 'sort_join', 'hash_join']
print_for_dataset("fb", algos, 1)
print_for_dataset("fb", algos, 10)
print_for_dataset("fb", algos, 100)
print_for_dataset("fb", algos, 1000)
#print_for_dataset("wiki", algos)
#print_for_dataset("osm", algos)
#print_for_dataset("books", algos)



## fb
|   threads |   hash_join |   inlj_btree256 |   lsj_sampledflatpgm256 |   sort_join |
|----------:|------------:|----------------:|------------------------:|------------:|
|         1 |     80.9313 |         49.3887 |                32.6124  |     31.2993 |
|         2 |     56.4348 |         37.4486 |                15.2571  |     29.4781 |
|         4 |     37.0868 |         19.6677 |                 9.97797 |     22.5739 |
|         6 |     31.0395 |         27.8541 |                 8.46116 |     20.3859 |
|         8 |     27.7622 |         24.0784 |                 7.71767 |     23.7554 |
## fb
|   threads |   hash_join |   inlj_btree256 |   lsj_sampledflatpgm256 |   sort_join |
|----------:|------------:|----------------:|------------------------:|------------:|
|         1 |    55.2928  |         8.00319 |                 7.77427 |     8.7249  |
|         2 |    28.0727  |         7.32919 |                 4.05965 |     7.20279 |
|         4 |    14.295   |         6.8212

In [24]:
def print_eps_vs_duration(dataset, algos, ratio):
    print("## " + dataset)
    algo_filter = ''
    for algo in algos:
        algo_filter = algo_filter + f" OR algo='{algo}'"
    algo_filter = algo_filter[3:]
    rows = duckdb.sql(
        " SELECT threads, ratio, epsilon, algo, MEDIAN(duration_sec) as v FROM df"
        f" WHERE dataset='{dataset}' AND ratio={ratio}"
        f" AND ({algo_filter})"
        " GROUP BY dataset, threads, ratio, algo, threads, epsilon"
        " ORDER BY ratio, threads"
    ).df()
    #print(rows)
    rows = rows.pivot(index='epsilon', values='v', columns='threads')
    print(rows.to_markdown())
    rows.to_csv('/home/chesetti/Repos/LearnedJoinDiskExp/experiments/results/join_eps_vs_duration/'+str(ratio)+'.txt')



algos = ['lsj_sampledflatpgm256', 'lsj_sampledflatpgm1024', 'lsj_sampledflatpgm4096']
print_eps_vs_duration("fb", algos, 1)
print_eps_vs_duration("fb", algos, 10)
print_eps_vs_duration("fb", algos, 100)
print_eps_vs_duration("fb", algos, 1000)
#print_for_dataset("wiki", algos)
#print_for_dataset("osm", algos)
#print_for_dataset("books", algos)


## fb
|   epsilon |        1 |       2 |        4 |       6 |       8 |
|----------:|---------:|--------:|---------:|--------:|--------:|
|       256 |  32.6124 | 15.2571 |  9.97797 | 8.46116 | 7.71767 |
|      1024 | nan      | 15.4888 | 10.1069  | 8.4592  | 7.75957 |
|      4096 | nan      | 16.4201 | 10.2662  | 8.61596 | 8.40002 |


KeyError: "None of [Index(['lsj_sampledflatpgm256', 'lsj_sampledflatpgm1024',\n       'lsj_sampledflatpgm4096'],\n      dtype='object', name='threads')] are in the [columns]"